In [47]:
import numpy as np
import pandas as pd
import xarray as xr
import scipy as sp
import scipy.optimize as opt
import scipy.stats as st
import matplotlib.pyplot as plt
import multiprocessing as mp
import numba as nb
import rasterio
import rioxarray as rxr
import lmoments3 as lm
from lmoments3 import distr
import time
#%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.family'] = ['SimSun']

In [2]:
def read_precip():
    filename_lst = []
    base_dir = "../../outputs/precip-cutted/"
    for i in range(0,6):
        year = 2000 + i*3
        for j in range(1,37):
            month = j
            filename_lst.append(base_dir + "_" + str(year) + "_" +str(year+2)+"_"+ str(month) + ".tif")
    filename_lst = filename_lst[0:192]
    
    data_frame = []
    for i in filename_lst:
        print(i)
        test_precip_frame = xr.open_dataset(i).squeeze("band",drop=True)
        test_precip_frame_resampled = test_precip_frame.rio.reproject(test_precip_frame.rio.crs,shape=(96,84),resampling=rasterio.enums.Resampling.bilinear)
        data_frame.append((test_precip_frame_resampled.band_data*0.1).to_numpy())
    return np.array(data_frame)

def read_pet():
    filename_lst = []
    base_dir = "../../outputs/pet-resampled/"
    for i in range(2000,2016):
        year = i
        for j in range(1,13):
            month = j
            filename_lst.append(base_dir + str(year) + "_" + str(month).zfill(2) + ".tif")

    data_frame = []
    for i in filename_lst:
        print(i)
        test_pet_frame = xr.open_dataset(i).squeeze("band",drop=True)
        test_pet_frame_resampled = test_pet_frame.rio.reproject(test_pet_frame.rio.crs,shape=(96,84),resampling=rasterio.enums.Resampling.bilinear)
        data_frame.append(test_pet_frame_resampled.band_data.to_numpy())
    return np.array(data_frame)


In [3]:
precip_monthly_mat=read_precip()
pet_monthly_mat=read_pet()

../../outputs/precip-cutted/_2000_2002_1.tif
../../outputs/precip-cutted/_2000_2002_2.tif
../../outputs/precip-cutted/_2000_2002_3.tif
../../outputs/precip-cutted/_2000_2002_4.tif
../../outputs/precip-cutted/_2000_2002_5.tif
../../outputs/precip-cutted/_2000_2002_6.tif
../../outputs/precip-cutted/_2000_2002_7.tif
../../outputs/precip-cutted/_2000_2002_8.tif
../../outputs/precip-cutted/_2000_2002_9.tif
../../outputs/precip-cutted/_2000_2002_10.tif
../../outputs/precip-cutted/_2000_2002_11.tif
../../outputs/precip-cutted/_2000_2002_12.tif
../../outputs/precip-cutted/_2000_2002_13.tif
../../outputs/precip-cutted/_2000_2002_14.tif
../../outputs/precip-cutted/_2000_2002_15.tif
../../outputs/precip-cutted/_2000_2002_16.tif
../../outputs/precip-cutted/_2000_2002_17.tif
../../outputs/precip-cutted/_2000_2002_18.tif
../../outputs/precip-cutted/_2000_2002_19.tif
../../outputs/precip-cutted/_2000_2002_20.tif
../../outputs/precip-cutted/_2000_2002_21.tif
../../outputs/precip-cutted/_2000_2002_22.t

In [4]:
d_monthly_mat = precip_monthly_mat - pet_monthly_mat
d_monthly_mat.shape

(192, 96, 84)

In [5]:
d_seq_00 = d_monthly_mat[:,0,0]

In [7]:
d_seq_00

array([ -98.9447  , -100.22266 , -201.74057 , -266.68866 , -289.90128 ,
       -311.46985 , -230.98956 , -215.64642 , -234.38318 , -190.17157 ,
       -155.46243 , -127.81737 , -105.372   , -116.209335, -224.4494  ,
       -263.73157 , -347.10825 , -293.00784 , -267.51926 , -235.75745 ,
       -243.88124 , -166.3352  , -142.21758 ,  -97.59447 , -121.00255 ,
       -121.97068 , -215.4927  , -212.0348  , -333.61523 , -225.55666 ,
       -255.0105  , -246.69699 , -177.2892  , -208.0623  , -159.8524  ,
       -106.02601 ,  -93.0156  , -107.42598 , -191.53716 , -241.76506 ,
       -240.14713 , -251.29263 , -198.60811 , -238.87523 , -199.56436 ,
       -162.70819 , -129.6269  , -104.39006 , -110.151474, -113.20167 ,
       -206.84976 , -285.8732  , -271.99518 , -229.1451  , -209.45245 ,
       -192.77087 , -220.57275 , -208.02585 , -162.70015 , -110.91189 ,
       -109.036606,  -73.40257 , -234.1589  , -284.26923 , -243.38554 ,
       -301.27496 , -253.88643 , -237.0181  , -236.9998  , -192.

In [58]:
def fisk(x,kappa,mu,lambda_):
    up = mu*kappa*np.float_power((lambda_*(x-mu)),(kappa-1))
    down = (1+np.float_power((lambda_*(x-mu)),kappa))**2
    return up/down

In [45]:
start = time.time()
para = sp.stats.fisk.fit(d_seq_00,method="MLE")
end = time.time()
print(end-start)

0.09721970558166504


In [ ]:
para

(0.3337619049000061, -347.10827036011744, 1.6076276098388542)

In [57]:
fisk(d_seq_00,*para)

array([-3.05155755e-02, -3.07137925e-02, -5.94692495e-02, -1.23354502e-01,
       -1.86796706e-01, -3.30365210e-01, -7.85317719e-02, -6.73617029e-02,
       -8.14597227e-02, -5.40750209e-02, -4.21632552e-02, -3.56295669e-02,
       -3.15362612e-02, -3.34017178e-02, -7.33920364e-02, -1.18019788e-01,
       -8.01306687e+04, -1.99874523e-01, -1.24931499e-01, -8.27016517e-02,
       -9.07937506e-02, -4.53490611e-02, -3.87876676e-02, -3.03086106e-02,
       -3.42907072e-02, -3.44753971e-02, -6.72643195e-02, -6.51384056e-02,
       -1.03691473e+00, -7.42192193e-02, -1.04465151e-01, -9.39374591e-02,
       -4.90210652e-02, -6.28403428e-02, -4.33999871e-02, -3.16435295e-02,
       -2.96250392e-02, -3.18753580e-02, -5.46654799e-02, -8.85529101e-02,
       -8.69055826e-02, -9.95080591e-02, -5.79164659e-02, -8.56487175e-02,
       -5.83827291e-02, -4.42397879e-02, -3.60008361e-02, -3.13764107e-02,
       -3.23354748e-02, -3.28646285e-02, -6.21676013e-02, -1.71986330e-01,
       -1.34092090e-01, -